## Notes:
- Doesnt appear to be training very well, training acc rises very slowly
- Needs to be tuned

Again will only run on Colab because of the loop moving memory onto the GPU

In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from torch.utils.data import DataLoader
from torch.autograd import Variable

### Data Loading

In [2]:
!git clone -b @olly-branch "https://github.com/BSteiner1/ST311-Group-Project/"

Cloning into 'ST311-Group-Project'...
remote: Enumerating objects: 2168, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 2168 (delta 76), reused 132 (delta 42), pack-reused 1998
Receiving objects: 100% (2168/2168), 1.20 GiB | 26.99 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Updating files: 100% (2019/2019), done.


In [3]:
class_dict = {'blues' : 0,
              'classical': 1,
              'country': 2,
              'disco' : 3,
              'hiphop' : 4,
              'jazz' : 5,
              'metal' : 6,
              'pop' : 7,
              'reggae': 8,
              'rock' : 9    
}

In [109]:
def collect_data():
    data = []
    # Collect all images from the 10 genres
    for genre in os.listdir("ST311-Group-Project/Data/images_original"):
        for image in os.listdir("ST311-Group-Project/Data/images_original/" + genre):
            image_path = "ST311-Group-Project/Data/images_original/" + genre + "/" + image
            grayscale_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            grayscale_img = grayscale_img[35:-36:, 54:-43]
            grayscale_img = torch.tensor(grayscale_img).to(dtype = torch.float32).clone().detach().requires_grad_(True)  #.requires_grad(True)
            #grayscale_img = torch.tensor(grayscale_img, requires_grad=True)
            # Splitting each image vertically into 5 different parts
            for i in range(5):
                # The data loader adds a dimension so I've removed a dimension from the image and class
                data.append((grayscale_img[ : ,  (67*i):67*(i+1)].reshape(1,217,67), torch.tensor(class_dict.get(genre)))) #.reshape(1)))
            
            #data.append((grayscale_img.reshape(1, 217, 335), torch.tensor(class_dict.get(genre))))
    return data     

data = collect_data()

In [110]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [230]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

CNN

In [231]:
'''
Net

'''

net3 = nn.Sequential(
    
    nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = "same"), nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = "same"), nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = "same"), nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(64,128, kernel_size = 3, stride = 1, padding = "same"), nn.ReLU(),

    nn.MaxPool2d(kernel_size = 4, stride = 4),

    nn.Conv2d(128, 64, kernel_size = 3, stride = 1, padding = "same"), nn.ReLU(),

    nn.Flatten(),

    nn.Linear(768, 384), nn.Dropout(0.2), nn.ReLU(),

    nn.Linear(384, 64), nn.Dropout(0.2), nn.ReLU(),

    nn.Linear(64,32), nn.Dropout(0.2), nn.ReLU(),

    nn.Linear(32,10), nn.ReLU(),

)

RNN

In [232]:
'''
Parameters
'''
# batch_size = 20
# n_iters = 3000
# num_epochs = n_iters / (len(train_dataset) / batch_size)
# num_epochs = int(num_epochs)

'''
Net
'''

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()

        #Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        #Embedding Layer
        #self.embed = nn.Embedding(10, 2)

        #Hidden dimensions
        self.hidden_dim = hidden_dim

        #Number of hidden layers
        self.layer_dim = layer_dim

        #RNN Layer 
        self.rnn = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional = True)

        #Linear Layer
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x):

        #Initialize hidden state with zeros
        # to.('cuda') ensures the hidden and input tensors are GPU

        x = self.pool(x)

        #x = self.embed(x)

        #print(out.size())

        h0 = torch.zeros(self.layer_dim*2, x.size(0), self.hidden_dim).requires_grad_().to('cuda')

        #RNN Foward Step
        out, hn = self.rnn(x, h0.detach())

        #print(out.size())

        #Clips off final hidden state 
        out = self.fc(out[:, -1, :]) 

        #print(out.size())

        return out

In [233]:
input_dim = 33
hidden_dim = 64
layer_dim = 2
output_dim = 10

rnn_model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim).to('cuda')

### Ensemble (Acc. 56%)

Net

In [234]:
class cnn_rnn(nn.Module):
    def __init__(self):
        super(cnn_rnn, self).__init__()

        #Net to process images
        self.net3 = net3

        #Net to process features
        self.rnn = rnn_model

        self.fc1 = nn.Linear(20, 10)

    def forward(self, x):

        #Pass image through cnn
        out1 = self.net3(x)

        #print(x.size())

        #Reshape for RNN
        seq_dim = 217
        input_dim = 67 

        x_rnn = x.view(-1, seq_dim, input_dim).requires_grad_()

        #Pass features through rnn
        out2 = self.rnn(x_rnn)

        #print(out2.size())

        #Concatenate
        concat = torch.cat((out1, out2), dim = 1)

        #print(concat.size())
        
        #Pass through linear layer
        out = self.fc1(concat)

        return out

Training

In [235]:
#Define instance of ensemble

cnn_rnn = cnn_rnn().to('cuda')

In [236]:
#Define Loss and Optimiser

cnn_rnn_optimizer = torch.optim.Adam(cnn_rnn.parameters(), lr=0.001)
cnn_rnn_loss = nn.CrossEntropyLoss()

In [237]:
#Define Epochs 

cnn_rnn_epochs = 20

In [238]:
#Main Loop

for epoch in range(cnn_rnn_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
      
        #Moves inputs to the gpu
        inputs = inputs.to('cuda')
        targets = targets.to('cuda')

        # Zero the gradients
        cnn_rnn_optimizer.zero_grad()
        
        # Forward pass
        outputs = cnn_rnn(inputs)
        loss = cnn_rnn_loss(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        cnn_rnn_optimizer.step()
        
        # Compute running loss and accuracy
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    # Compute validation loss and accuracy
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:

            #Moves inputs to gpu
            inputs = inputs.to('cuda')
            targets = targets.to('cuda')
            
            outputs = cnn_rnn(inputs)
            loss = cnn_rnn_loss(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
    
    # Print epoch statistics
    print('Epoch %d: Train Loss: %.3f | Train Acc: %.3f%% | Val Loss: %.3f | Val Acc: %.3f%%'
          % (epoch+1, running_loss/len(train_loader), 100*correct/total,
             val_loss/len(test_loader), 100*val_correct/val_total))

RuntimeError: ignored

### Sequential CNN-RNN

In [199]:
x1 = torch.zeros(16,1,217,67)

In [200]:
x1.size()

torch.Size([16, 1, 217, 67])

RuntimeError: ignored